#### As usual, there are two method
- [Core method](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html#tutorial-core-insert)
- [ORM method](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)

#### here illustrated orm method

#### refresher/prerequisite to the orm method
- [Executing with an ORM Session - introduces how to make an ORM Session object](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)
- [Using ORM Declarative Forms to Define Table Metadata - where we set up our ORM mappings of the User and Address entities](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-orm-table-metadata)
- [Selecting ORM Entities and Columns - a few examples on how to run SELECT statements for entities like User](https://docs.sqlalchemy.org/en/20/tutorial/data_select.html#tutorial-selecting-orm-entities)

In [1]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey
from sqlalchemy.orm import Session
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

class User(Base):
    
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"
    
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"



engine_config = template_engine.format(
        db_username="a",
        db_password="v",
        db_host="c.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="")


engine = create_engine(engine_config,
    echo=True
) 


## select

In [5]:
from sqlalchemy import select

with Session(engine) as session:

    #select with filter
    users = session.execute(select(User))
    
    for user in users:
        print(user)

2023-03-06 22:03:54,663 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 22:03:54,666 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-03-06 22:03:54,668 INFO sqlalchemy.engine.Engine [generated in 0.00165s] {}
(User(id=14, name='squidward', fullname='Squidward Tentacles'),)
(User(id=16, name='squidward', fullname='Squidward Tentacles'),)
(User(id=17, name='ehkrabs', fullname='Eugene H. Krabs'),)
(User(id=15, name='ehkrabs', fullname='Pegasus Extraordinaire'),)
2023-03-06 22:03:54,693 INFO sqlalchemy.engine.Engine ROLLBACK


In [42]:
from sqlalchemy import select

with Session(engine) as session:

    #select with filter
    users = session.execute(select(User).where(User.name == 'ehkrabs').where(User.fullname == 'Pegasus Extraordinaire'))
    
    #print(users.all())
    
    for user in users:
        print("----")
        print(type(user))
        print(user[0].id)


2023-03-06 22:36:42,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 22:36:42,429 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s::VARCHAR(30) AND user_account.fullname = %(fullname_1)s::VARCHAR
2023-03-06 22:36:42,431 INFO sqlalchemy.engine.Engine [cached since 1695s ago] {'name_1': 'ehkrabs', 'fullname_1': 'Pegasus Extraordinaire'}
----
<class 'sqlalchemy.engine.row.Row'>
15
2023-03-06 22:36:42,447 INFO sqlalchemy.engine.Engine ROLLBACK
